In [14]:
import torch
from torch import nn
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import torch.optim as optim

In [15]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [16]:
# Load data (one large matrix?)
datafiles = ['../project/dataset1.csv', '../project/dataset2.csv', '../project/dataset3.csv', '../project/dataset4.csv', '../project/dataset5.csv']
n_samples = 50000           # Number of samples
n_dataset = len(datafiles)  # Number of datasets
train = 0.9   # 245000 (77.777...%)
test = 0.1    # 35000 (10%)
# val = 1-train # 70000 (Rest)

size_test = int(n_samples*n_dataset*test)+1
size_train = int((n_samples*n_dataset-size_test)*train)+1
# size_val = int((n_samples*n_dataset-size_test)*val)+1

In [17]:
# Split all datasets
datasets = np.empty((int(n_samples*n_dataset),5))
print(datasets.shape)
for i,datafile in enumerate(datafiles):
  dataset = np.loadtxt(datafile, delimiter=',')
  datasets[int(i*n_samples):int(n_samples*(i+1)),:] = dataset
print(datasets[0,0])
# Split into input and output
Cr = datasets[:,0:4]   # Input: relative entropy of coherence
ES = np.array([float(y) for y in datasets[:,4]])  # Output: entangled/separable

# Count number of entangled and separable states
n_entangled = len([es for es in ES if es == 1])
print(f'Number of entangled states: {n_entangled}\n')
print(f'Number of separable states: {n_samples*n_dataset-n_entangled}\n')
# Split into train-and-validation and test
#break datasets into train and test

Cr_train, Cr_test, ES_train, ES_test = train_test_split(Cr, ES, test_size=test,
                                                      random_state=42)
# # Split into train and validation
# Cr_train, Cr_val, ES_train, ES_val = train_test_split(Cr_trainval, ES_trainval, test_size=val,
#                                                       random_state=30)
# DEBUG
print(np.shape(Cr_train))
print(np.shape(Cr_test))
print(np.shape(ES_train))
print(np.shape(ES_test))

(250000, 5)
0.195132062642601
Number of entangled states: 127330

Number of separable states: 122670

(225000, 4)
(25000, 4)
(225000,)
(25000,)


In [18]:
# Tansform data into tensors
Cr_tensor = torch.tensor(Cr, dtype=torch.float32)
ES_tensor = torch.tensor(ES, dtype=torch.float32)
# Cr_train = torch.tensor(Cr_train, dtype=torch.float32)
# Cr_test = torch.tensor(Cr_test, dtype=torch.float32)
# ES_train = torch.tensor(ES_train, dtype=torch.float32)
# ES_test = torch.tensor(ES_test, dtype=torch.float32)

# # In case we have a gpu
Cr_tensor = Cr_tensor.to(device)
ES_tensor = ES_tensor.to(device)
# Cr_train = Cr_train.to(device)
# Cr_test = Cr_test.to(device)
# ES_train = ES_train.to(device)
# ES_test = ES_test.to(device)

In [19]:
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset, DataLoader
tensor_dataset = TensorDataset(Cr_tensor, ES_tensor)
dataset = DataLoader(tensor_dataset, batch_size=32, shuffle=True)

In [20]:
# Define the neural network architecture
class NeuralNetwork_3layer_with_a0(nn.Module):
    def __init__(self, n_nodes):
        super().__init__()
        self.layer1 = nn.Linear(4, n_nodes)
        self.layer2 = nn.Linear(n_nodes,4)
        self.layer3 = nn.Linear(4, 1)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()
        self.apply(self._init_weights)


    def forward(self, x):
      # if input x provided, describe how to produce output tensor
        x = self.relu(self.layer1(x))
        x = self.sig(self.layer2(x))
        x = self.sig(self.layer3(x))
        return x

    def _init_weights(self, l):
      # Initialize weights uniformly
      if isinstance(l, nn.Linear):
        nn.init.kaiming_uniform_(l.weight)
        #nn.init.uniform_(l.weight)
        l.bias.data.fill_(0.01)

In [21]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
        self.bce_loss = nn.BCELoss()

    def forward(self, predicted, target, weights, input, offset):
        bce_loss = self.bce_loss(predicted, target)
        additional_loss = torch.sum(weights*input) + offset
        if additional_loss <= 4:
            additional_loss = 0
        else:
            additional_loss = 1

        total_loss = bce_loss + additional_loss
        return total_loss

In [22]:
def train_model(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Train the model
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction error
        y = y.view(-1, 1)
        pred = model(X)
        weights = list(model.layer3.parameters())[0].data[0]
        offset = list(model.layer3.parameters())[1].data[0]
        loss = loss_fn(pred, y, weights, X, offset)
        # loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_model(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            # Reshape y to match pred's shape
            y = y.view(-1, 1)  # Assuming y has shape (batch_size, 1)
            pred = model(X)
            weights = list(model.layer3.parameters())[0].data[0]
            offset = list(model.layer3.parameters())[1].data[0]
            test_loss += loss_fn(pred, y, weights, X, offset).item()
            # test_loss += loss_fn(pred, y).item()
            correct += (pred.round() == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
model = NeuralNetwork_3layer_with_a0(50).to(device)
loss_fn = CustomLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_model(dataset, model, loss_fn, optimizer)
    test_model(dataset, model, loss_fn)

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 86.3%, Avg loss: 17.705277 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 38.416301 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 91.1%, Avg loss: 56.094977 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 67.231701 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 95.0%, Avg loss: 74.307088 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 93.5%, Avg loss: 80.147716 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 94.1%, Avg loss: 84.137206 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 95.1%, Avg loss: 87.084696 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 95.8%, Avg loss: 90.144132 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 93.9%, Avg loss: 92.306541 

Epoch 11
-------------------------------
Test Error: 
 Accuracy: 95.2

In [24]:
Cr = datasets[:,0:4]
test = torch.tensor(Cr, dtype=torch.float32)
print(test[0])
print(ES_tensor[2])

tensor([0.1951, 0.2041, 0.1951, 0.0946])
tensor(0.)


In [25]:
# Get the model parameters of the last layer
weights = list(model.layer3.parameters())[0].data[0]
offset = list(model.layer3.parameters())[1].data[0]


In [26]:
print(weights)
print(offset)

tensor([  8.2876, -14.6224,   9.3816,  10.7306])
tensor(-4.0716)


In [27]:
correct_predictions = 0
total_predictions = len(ES_tensor)
predictions_of_0 = 0
correct_predictions_of_0 = 0
predictions_of_1 = 0
correct_predictions_of_1 = 0

for i in range(total_predictions):
    result = torch.sum(weights * test[i]) + offset
    if result > 4:
        result = 1
    else:
        result = 0

    # Compare with expected value
    if result == ES_tensor[i]:
        correct_predictions += 1
    if ES_tensor[i] == 0:
        predictions_of_0 += 1
        if result == 0:
            correct_predictions_of_0 += 1
    if ES_tensor[i] == 1:
        predictions_of_1 += 1
        if result == 1:
            correct_predictions_of_1 += 1

# Calculate accuracy
accuracy = correct_predictions / total_predictions * 100
print(f'Accuracy: {accuracy}%')
# Calculate accuracy for 0
accuracy_0 = correct_predictions_of_0 / predictions_of_0 * 100
print(f'Accuracy for 0: {accuracy_0}%')
# Calculate accuracy for 1
accuracy_1 = correct_predictions_of_1 / predictions_of_1 * 100
print(f'Accuracy for 1: {accuracy_1}%')

Accuracy: 50.3392%
Accuracy for 0: 84.38901116817478%
Accuracy for 1: 17.535537579517786%
